In [3]:
import os
import pandas as pd
import numpy as np
import json
from datetime import datetime
from dotenv import load_dotenv

# --- Fyers API Import ---
from fyers_apiv3 import fyersModel

# --- LangChain Imports ---
import langchain
from langchain_groq import ChatGroq
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_experimental.tools import PythonREPLTool

# --- 1. Load Environment Variables ---
load_dotenv()

GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
CLIENT_ID = os.environ.get("FYERS_CLIENT_ID")
ACCESS_TOKEN = os.environ.get("FYERS_TOKEN")

print(f"{GROQ_API_KEY}\n{CLIENT_ID}\n{ACCESS_TOKEN}\n\n")

ImportError: cannot import name 'AgentExecutor' from 'langchain.agents' (c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\agents\__init__.py)

In [ ]:
from fyers_apiv3 import fyersModel

# Use your existing access_token
fyers = fyersModel.FyersModel(token=access_token, is_async=False, client_id=client_id, log_path="")

# Example 1: Get Stock Quote (Real-time price data)
symbol = "NSE:RELIANCE-EQ"     # Format: EXCHANGE:SYMBOL
data = {
    "symbols": symbol
}
quote = fyers.quotes(data)
print("Real-Time Quote:\n", quote)

# Example 2: Get Market Depth (Order Book Data)
depth = fyers.depth(data)
print("\nMarket Depth:\n", depth)

# Example 3: Get Historical Data (Candlestick / OHLC)
historical_data = {
    "symbol": symbol,
    "resolution": "D",      # D=Daily, 5=5min, 15=15min, 60=1hr
    "date_format": "1",     # 1=YYYY-MM-DD
    "range_from": "2024-01-01",
    "range_to": "2024-12-31",
    "cont_flag": "1"        # Continuous data for expired contracts
}
history = fyers.history(data=historical_data)
print("\nHistorical Data:\n", history)


In [ ]:
@tool
def get_fyers_data(symbol: str, start_date: str, end_date: str) -> str:
    """
    Fetches historical daily OHLCV data from Fyers (API v3) for a given symbol,
    saves it as CSV, and returns a status message.
    """
    print(f"--- Tool Call: Fetching data for {symbol} ---")

    if not (CLIENT_ID and ACCESS_TOKEN):
        return "❌ Error: Missing FYERS_CLIENT_ID or FYERS_ACCESS_TOKEN in environment."

    try:
        fyers = fyersModel.FyersModel(
            token=ACCESS_TOKEN,
            client_id=CLIENT_ID,
            is_async=False,
            log_path=""
        )

        data = {
            "symbol": symbol,
            "resolution": "D",
            "date_format": "1",
            "range_from": start_date,
            "range_to": end_date,
            "cont_flag": "1"
        }

        response = fyers.history(data=data)

        if response.get("s") != "ok" or "candles" not in response:
            return f"❌ Error fetching data: {response.get('message', 'Unknown Fyers error')}"

        df = pd.DataFrame(response["candles"], columns=[
            "timestamp", "open", "high", "low", "close", "volume"
        ])

        if df.empty:
            return f"⚠️ No data returned for {symbol}."

        df["datetime"] = pd.to_datetime(df["timestamp"], unit="s")
        df.set_index("datetime", inplace=True)
        df.rename(columns={
            "open": "Open",
            "high": "High",
            "low": "Low",
            "close": "Close",
            "volume": "Volume"
        }, inplace=True)
        df = df[["Open", "High", "Low", "Close", "Volume"]]

        file_name = f"{symbol.replace(':', '_')}.csv"
        df.to_csv(file_name)

        return f"✅ Successfully fetched {len(df)} rows for {symbol} and saved to '{file_name}'."

    except Exception as e:
        return f"❌ Exception in get_fyers_data: {str(e)}"


print("Tool defined")

# --- 2. Initialize Groq LLM ---
llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0,
    api_key=GROQ_API_KEY
)

# --- 3. Python Execution Tool ---
python_tool = PythonREPLTool()
tools = [get_fyers_data, python_tool]
print("Interpreter defined")


# --- 4. Create the Agent ---

def create_backtesting_agent():
    """
    Creates and returns a LangChain AgentExecutor capable of fetching data
    and running Backtrader strategies, printing metrics and code.
    """

    prompt_template = """
    You are an expert quantitative analyst AI. You have two tools:
    1. get_fyers_data: Fetches data and saves it to CSV.
    2. python_repl: Executes Python code.

    Your task: Backtest the trading strategy described by the user.

    You MUST follow this process:
    Step 1: Fetch data using `get_fyers_data`.
    Step 2: Write and execute ONE complete Python code block for backtesting.

    The code you write MUST:
    - Import backtrader (bt), pandas (pd), and math/numpy.
    - Read the CSV: `data = bt.feeds.PandasData(dataname=pd.read_csv('<symbol>.csv', index_col='datetime', parse_dates=True))`
    - Define a Backtrader strategy class with the users logic.
    - Initialize Cerebro, add data and strategy.
    - Set starting cash: 100000.
    - Run the backtest.
    - Compute and print metrics:
        * Starting and Final Portfolio Value
        * Total Return %
        * CAGR %
        * Max Drawdown %
        * Sharpe Ratio
        * Number of Trades, Win Rate %, Average Win/Loss
    - Print the entire Python code you used before executing (for transparency).

    Example metric calculations:
        returns = cerebro.run()
        pnl = cerebro.broker.getvalue() - start_cash
        total_return = (cerebro.broker.getvalue() / start_cash - 1) * 100
        sharpe = (mean_daily_return / std_daily_return) * sqrt(252)
        drawdown analyzer = cerebro.addanalyzer(bt.analyzers.DrawDown)
        trade analyzer = cerebro.addanalyzer(bt.analyzers.TradeAnalyzer)

    Do NOT omit any print statements. Show everything clearly.

    User's Request:
    {input}

    Agent Scratchpad:
    {agent_scratchpad}
    """

    prompt = ChatPromptTemplate.from_template(prompt_template)
    agent = create_tool_calling_agent(llm, tools, prompt)

    return AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True
    )


# --- 5. Run the Agent ---

def main():
    """
    Main function to run the chatbot agent.
    """
    if not (GROQ_API_KEY and CLIENT_ID and ACCESS_TOKEN):
        print("❌ Error: Missing keys in environment.")
        return

    print("--- Backtesting Agent Initialized ---")
    print(" Note: Python REPL runs arbitrary code. Avoid production use.")

    agent_executor = create_backtesting_agent()

    user_request = (
        "Please backtest a 10/30 SMA crossover on 'NSE:TCS-EQ' "
        "from '2024-01-01' to '2024-05-31'. "
        "Buy when fast SMA crosses above slow SMA and sell on reverse cross."
    )

    print(f"\n--- Sending Request ---\n{user_request}\n")

    try:
        response = agent_executor.invoke({"input": user_request})
        print("\n--- Final Chatbot Response ---")
        print(response["output"])
    except Exception as e:
        print(f"\n❌ Agent execution failed: {e}")


# --- Entry Point ---
if __name__ == "__main__":
    # Required packages:
    # pip install langchain langchain-groq langchain-experimental fyers-apiv3 backtrader python-dotenv pandas
    main()


Tool defined
Interpreter defined
--- Backtesting Agent Initialized ---
⚠️ Note: Python REPL runs arbitrary code. Avoid production use.

--- Sending Request ---
Please backtest a 10/30 SMA crossover on 'NSE:TCS-EQ' from '2024-01-01' to '2024-05-31'. Buy when fast SMA crosses above slow SMA and sell on reverse cross.



> Entering new AgentExecutor chain...

Invoking: `get_fyers_data` with `{'end_date': '2024-05-31', 'start_date': '2024-01-01', 'symbol': 'NSE:TCS-EQ'}`


--- Tool Call: Fetching data for NSE:TCS-EQ ---
✅ Successfully fetched 104 rows for NSE:TCS-EQ and saved to 'NSE_TCS-EQ.csv'.
Invoking: `Python_REPL` with `{'query': "# Backtesting 10/30 SMA crossover for NSE:TCS-EQ\nimport backtrader as bt\nimport pandas as pd\nimport numpy as np\nimport math\n\n# Load data\ncsv_file = 'NSE_TCS-EQ.csv'\n# Ensure datetime is parsed and set as index\ndf = pd.read_csv(csv_file, parse_dates=True, index_col='datetime')\n# Backtrader expects column names: open, high, low, close, volume\n# If 